In [1]:
import pandas
import networkx as nx

Trabajamos para una discógráfica importante; nuestra misión corporativa es <strike>[rent seeking](https://en.wikipedia.org/wiki/Rent-seeking)</strike> proteger derechos de autor.

Nuestros [colaboradores](https://en.wikipedia.org/wiki/Unofficial_collaborator) en distintas ciudades nos mandan informes de cuándo escuchan una canción por la calle. También disponemos de una base de datos interna que asocia canciones a identificadores [iswc](https://en.wikipedia.org/wiki/International_Standard_Musical_Work_Code).

Necesitamos combinar los reportes de campo con nuestra base de datos interna vía [data fusion](https://en.wikipedia.org/wiki/Data_fusion) para poder obtener un conjunto de datos desduplicado.

* [Los datos](#los-datos)
    * [reference.csv](#reference)
    * [field_reports.csv](#field-reports)
* [El resultado](#el-resultado)
* [El algoritmo](#el-algoritmo)
* [Optimizar el algoritmo](#optimizar-el-algoritmo)

### Los datos<a class="anchor" id="los-datos"></a>

#### reference.csv<a class="anchor" id="reference"></a>

[reference.csv](reference.csv) asocia canciones a identificadores [iswc](https://en.wikipedia.org/wiki/International_Standard_Musical_Work_Code); una canción puede tener varios identificadores distintos.

In [2]:
reference = pandas.read_csv('reference.csv')
reference

,id,iswc,title,participants
0,0,T-800.802.619-6,Du Hast,kruspe
1,1,T-800.802.619-6,Du Hast,kruspe|lindemann
2,2,T-800.125.458-7,Bück dich,kruspe|lindemann
3,3,T-071.081.651-5,Dope Man,Ice Cube|Dr. Dre
4,4,T-070.044.219-8,Dope Man,Ice Cube


#### field_reports.csv<a class="anchor" id="field-reports"></a>

Field reports asocia títulos de canciones y participantes a sitios donde se ha escuchado esa canción; los trabajadores de campo no se saben los iswcs así que apuntan el título y los artistas que recuerdan.

In [3]:
field_reports = pandas.read_csv('field_reports.csv')
field_reports

,id,title,participants,place
0,0,Du Hast,kruspe,valencia
1,0,Du Hast,kruspe,berlin
2,1,Bück dich,kruspe|lindemann|landers,berlin
3,2,Dope Man,Ice Cube|Dr. Dre,massalfassar
4,3,Space Oddity,Bowie,valencia
5,4,Dope Man,Ice Cube|Dr. Dre,valencia


### El resultado<a class="anchor" id="el-resultado"></a>

Queremos combinar estos csvs para generar una tabla desduplicada con los siguientes campos, con una fila por canción:

* **title**: el título de la canción
* **iswcs**: todos los iswc asociados a esta canción, separados por `|`
* **places**: todas las ciudades donde se ha escuchado esta canción según los reportes de campo, separados por `|`

### El algoritmo<a class="anchor" id="el-algoritmo"></a>

Para desduplicar las filas podemos usar las siguientes relaciones:

* si 2 canciones tienen el mismo iswc, consideramos que son la misma canción
* si 2 canciones tienen el mismo `title` y comparten al menos 1 `participant`, consideramos que son la misma canción

Nuestro data scientist ha desarrollado un algoritmo (lento) que agrupa las filas para encontrar las canciones de la siguiente manera:

1. construye un grafo donde los nodos son las filas de los conjuntos de datos de `reference.csv` y `old_data.csv`
2. conecta los nodos que tienen la relación 1 (iswc igual) con edges no dirigidas
3. conecta los nodos que tienen la relación 2 (title y al menos 1 colaborador compartido) con edges no dirigidas
4. encuentra las [componentes conexas](https://en.wikipedia.org/wiki/Component_(graph_theory)) de este grafo: cada componente conexa es una canción

In [4]:
def split_participants(participants):
    return set(participants.split('|'))

def load_reference(g, df):
    for r in df.to_dict(orient="records"):
        key = ('reference.csv', r['id'])
        g.add_node(key,
            id=r['id'],
            iswc=r['iswc'],
            title=r['title'],
            participants=split_participants(r['participants'])
        )

def load_field_reports(g, df):
    for r in df.to_dict(orient="records"):
        key = ('field_reports.csv', r['id'])
        g.add_node(key,
            id=r['id'],
            title=r['title'],
            participants=split_participants(r['participants']),
            place=r['place']
        )
        
def add_iswc_edges(g):
    for n in g.nodes:
        for m in g.nodes:
            if n != m:
                n_iswc = g.nodes[n].get('iswc')
                m_iswc = g.nodes[m].get('iswc')
                if n_iswc and m_iswc and n_iswc == m_iswc:
                    g.add_edge(n, m)
                    
def add_title_participant_edges(g):
    # el coste de este algoritmo es O(n^2) en el número n de nodos del grafo
    for n in g.nodes:
        for m in g.nodes:
            if n != m and g.nodes[n]['title'] == g.nodes[m]['title']:
                shared_participants = g.nodes[n]['participants'].intersection(g.nodes[m]['participants'])
                if len(shared_participants) > 0:
                    g.add_edge(n, m)
                    
                    

Una vez disponemos del grafo con todos los datos y edges cargados, podemos extraer las canciones identificando las componentes conexas y fusionando apropiadamente los datos de todas las filas de una componente nos dará el output deseado:

In [5]:
def component_to_song(c, g):
    titles = set()
    iswcs = set()
    places = set()

    for key in c:
        titles.add(g.nodes[key]['title'])
        
        iswc = g.nodes[key].get('iswc')
        if iswc:
            iswcs.add(iswc)
            
        place = g.nodes[key].get('place')
        if place:
            places.add(place)

        if len(titles) > 1:
            raise Exception("A song with multiple titles contradicts our assumptions")

    return dict(
        iswcs='|'.join(iswcs),
        title=list(titles)[0],
        places='|'.join(places),
    )

def extract_songs(g):
    records = [component_to_song(c, g) for c in nx.algorithms.components.connected_components(g)]    
    return pandas.DataFrame.from_records(records, columns=['title', 'iswcs', 'places'])
        

In [6]:
# create graph, load node data
g = nx.Graph()
load_reference(g, reference)
load_field_reports(g, field_reports)

# add relationship edges
add_iswc_edges(g)
add_title_participant_edges(g)

# use graph to fuse data
fused_data = extract_songs(g)
fused_data

,title,iswcs,places
0,Du Hast,T-800.802.619-6,berlin
1,Bück dich,T-800.125.458-7,berlin
2,Dope Man,T-070.044.219-8|T-071.081.651-5,valencia|massalfassar
3,Space Oddity,,valencia


### Optimizar el algoritmo<a class="anchor" id="optimizar-el-algoritmo"></a>

El algoritmo funciona bien pero es lento: si tenemos n filas en total combinando `reference.csv` y `field_reports.csv`,
el [coste del algoritmo](https://en.wikipedia.org/wiki/Big_O_notation) será del orden $O(n^2)$, ya que lleva a cabo comparaciones entre todos los nodos del grafo para determinar si existen relaciones entre ellos.

Para nuestros csvs de juguete no es un problema, pero si tuviésemos csvs con millones de canciones sería otro cantar.

#### 1 

**Encuentra un algoritmo más rápido**.

¿Puedes cambiar el algoritmo para que sea más eficiente? Da igual que uses networkx, o python puro, o otra cosa.

#### 2 

El tiempo de ejecución del algoritmo del data scientist depende del número total de filas en `reference.csv` y `field_reports.csv`: si hay `n` filas en total, el tiempo de ejecución será del órden $O(n^2)$

¿De qué depende el tiempo de ejecución de tu algoritmo?